# Data Format
This file formats the dataset for hand pose estimation

In [20]:
import json
import os
from PIL import Image
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import fnmatch
import re

## Constants

In [21]:
# finger and joints
fingers = ["thumb", "index", "middle", "ring", "pinky"]
joints = ["distal", "intermediate", "proximal", "metacarpal"]

# sequense length
n = 10

# define which video to use as test case
min_value = 0
max_value = 20
num_integers = 5

test = np.random.choice(np.arange(min_value, max_value), num_integers, replace=False)
test.sort()

# the path to the data
root = "../../../datasets/everyone/data/"

# all the video data folder
videos = os.listdir(root)

# export location
export_location = "./data/"

## Helper Function
for formatting data annotation

In [22]:
def readAnnotation(hand):
    an = []
    re_an = []
    root = np.array(hand["palm_position"])
    an.append(hand["palm_position"])
    re_an.append([0., 0., 0.])
    for finger in fingers:
        for joint in joints:
            an.append(hand[finger][joint])
            re_an.append((np.array(hand[finger][joint]) - root).tolist())
    
    return an, re_an

def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

def recursive_glob(rootdir=".", pattern="*"):
    matches = []
    for root, dirnames, filenames in os.walk(rootdir):
        for filename in fnmatch.filter(filenames, pattern):
            matches.append(os.path.join(root, filename))
    
    return matches

## Formatting Process

In [23]:
test_data = [f"data{i}" for i in test]

image_paths = []
reletive_hands = []
movements = []

image_paths_test = []
reletive_hands_test = []
movements_test = []

for video in videos:
    if os.path.isdir(os.path.join(root, video)) and "data" in video:
        colorFrames = recursive_glob(os.path.join(root, video, "ring"), "image[0-9]*")
        colorFrames = natural_sort(colorFrames)
        with open(os.path.join(root, video, "data_leap.json"), "r") as f:
            annotation = json.load(f)

        if video in test_data:
            for i in range(0, len(colorFrames), n):
                root_num = str(int(colorFrames[i].split('.')[6].split("image")[-1]) + (n - 2))
                root_pos = np.array(readAnnotation(annotation[root_num])[0][0])
                image_path = []
                reletive_hand = []
                movement = []
                for j in range(i, i+n):
                    num = colorFrames[j].split(".")[6].split("image")[-1]
                    image_path.append(colorFrames[j])
                    _an, re_an = readAnnotation(annotation[num])
                    reletive_hand.append(re_an)
                    an = []
                    for a in _an:
                        an.append((np.array(a) - root_pos).tolist())
                    movement.append(an)
                image_paths_test.append(image_path)
                reletive_hands_test.append(reletive_hand)
                movements_test.append(movement)
        else:
            for i in range(0, len(colorFrames), n):
                root_num = str(int(colorFrames[i].split('.')[6].split("image")[-1]) + (n - 2))
                root_pos = np.array(readAnnotation(annotation[root_num])[0][0])
                image_path = []
                reletive_hand = []
                movement = []
                for j in range(i, i+n):
                    num = colorFrames[j].split(".")[6].split("image")[-1]
                    image_path.append(colorFrames[j])
                    _an, re_an = readAnnotation(annotation[num])
                    reletive_hand.append(re_an)
                    an = []
                    for a in _an:
                        an.append((np.array(a) - root_pos).tolist())
                    movement.append(an)
                image_paths.append(image_path)
                reletive_hands.append(reletive_hand)
                movements.append(movement)

### Conver into numpy array

In [24]:
image_paths = np.array(image_paths)
reletive_hands = np.array(reletive_hands)
movements = np.array(movements)

image_paths_test = np.array(image_paths_test)
reletive_hands_test = np.array(reletive_hands_test)
movements_test = np.array(movements_test)

### Export to numpy files

In [25]:
np.save(export_location + "image_path.npy", image_paths)
np.save(export_location + "reletive_hands", reletive_hands)
np.save(export_location + "movement.npy", movements)

np.save(export_location + "image_path_test.npy", image_paths_test)
np.save(export_location + "reletive_hands_test", reletive_hands_test)
np.save(export_location + "movement_test", movements_test)